<a href="https://colab.research.google.com/github/sjmama/sem/blob/main/%EC%8A%AC%EB%9D%BC%EC%9D%B4%EB%94%A9_%EC%9C%88%EB%8F%84%EC%9A%B0_%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from dateutil.relativedelta import relativedelta
import numpy as np
import pandas as pd

In [ ]:
df_raw=pd.read_csv("shopping_mall.csv")

# 데이터 샘플 확인
df_raw.head(10)

,cust_id,dt,ym,sale_amt
0,AFG4127094465,2021-12-09,202112,300000
1,AFG4127094465,2022-04-22,202204,45000
2,AFG4136725677,2022-04-07,202204,47800
3,AFG4140129314,2022-02-07,202202,39200
4,AFG4144923657,2022-05-11,202205,110000
5,AFG4154711536,2022-02-19,202202,45000
6,AFG4155110728,2022-01-26,202201,141550
7,AFG4160618910,2022-04-02,202204,100000
8,AFG4168452546,2022-02-04,202202,183200
9,AFG4172879718,2022-05-08,202205,39200


In [ ]:
df_raw['Date2']= pd.to_datetime(df_raw['dt'])

# 시점변수 생성
ym_li = [f"YM_M{i}" for i in range(6)]
ym_li

for i in range(len(ym_li)):
    col = ym_li[i]
    now = pd.to_datetime('2022-05-01') + relativedelta(months=-i)
# 현재시점 기준 미래(-) 과거(+)
    df_raw[col] = 'M' + (-np.floor((df_raw.Date2 - now)
                                   /np.timedelta64(1, 'M'))).astype(int).astype(str)

df_raw.head(10)

,cust_id,dt,ym,sale_amt,Date2,YM_M0,YM_M1,YM_M2,YM_M3,YM_M4,YM_M5
0,AFG4127094465,2021-12-09,202112,300000,2021-12-09,M5,M4,M3,M2,M1,M0
1,AFG4127094465,2022-04-22,202204,45000,2022-04-22,M1,M0,M-1,M-2,M-3,M-4
2,AFG4136725677,2022-04-07,202204,47800,2022-04-07,M1,M0,M-1,M-2,M-3,M-4
3,AFG4140129314,2022-02-07,202202,39200,2022-02-07,M3,M2,M1,M0,M-1,M-2
4,AFG4144923657,2022-05-11,202205,110000,2022-05-11,M0,M-1,M-2,M-3,M-4,M-5
5,AFG4154711536,2022-02-19,202202,45000,2022-02-19,M3,M2,M1,M0,M-1,M-2
6,AFG4155110728,2022-01-26,202201,141550,2022-01-26,M4,M3,M2,M1,M0,M-1
7,AFG4160618910,2022-04-02,202204,100000,2022-04-02,M1,M0,M-1,M-1,M-2,M-4
8,AFG4168452546,2022-02-04,202202,183200,2022-02-04,M3,M2,M1,M0,M-1,M-2
9,AFG4172879718,2022-05-08,202205,39200,2022-05-08,M0,M-1,M-2,M-3,M-4,M-5


In [ ]:
# 슬라이딩 윈도우 형태로 변환

m_col = ["M{}".format(i) for i in range(6)]   # M0~M5 목록 생성
df_li = []   # 임시테이블 저장할 목록

for n, ym in enumerate(ym_li):  # YM_M0 ~ YM_M5 반복
# STD_YM_M0 변수 기준 M0~M5 & 구매금액 0원 초과
    tmp = df_raw[(df_raw[ym].isin(m_col)) & (df_raw['sale_amt'] > 0)]
# YM_M0 기준 pivot
    tmp = tmp.pivot_table(index='cust_id',
                          columns=ym, values='sale_amt',
                          aggfunc='sum')

    # M0~M12 중 누락된 컬럼 생성
    # 추후 테이블 union을 위해 pivot시 누락된 컬럼을 별도로 생성해줌
    missing_col = list(set(m_col) - set(tmp.columns))
    for col in missing_col :
        tmp[col] = 0

    # 컬럼이름 변경
    tmp.columns = [f'slae_amt_{c}' for c in tmp.columns] # 생략 가능

    tmp['MM_DIFF'] = ym
    tmp = tmp.fillna(0)

    df_li.append(tmp)

final_df = pd.concat(df_li).reset_index()
final_df.head(10)

,cust_id,slae_amt_M0,slae_amt_M1,slae_amt_M2,slae_amt_M3,slae_amt_M4,slae_amt_M5,MM_DIFF
0,AFG4127094465,0.0,45000.0,0.0,0.0,0.0,300000.0,YM_M0
1,AFG4136725677,0.0,47800.0,0.0,0.0,0.0,0.0,YM_M0
2,AFG4140129314,0.0,0.0,0.0,39200.0,0.0,0.0,YM_M0
3,AFG4144923657,110000.0,0.0,0.0,0.0,0.0,0.0,YM_M0
4,AFG4154711536,0.0,0.0,0.0,45000.0,0.0,0.0,YM_M0
5,AFG4155110728,0.0,0.0,0.0,0.0,141550.0,0.0,YM_M0
6,AFG4160618910,0.0,100000.0,0.0,0.0,0.0,0.0,YM_M0
7,AFG4168452546,0.0,0.0,0.0,183200.0,0.0,0.0,YM_M0
8,AFG4172879718,39200.0,0.0,0.0,0.0,0.0,0.0,YM_M0
9,AFG4174345172,23900.0,0.0,0.0,0.0,0.0,0.0,YM_M0
